# Deepfake Detection

In [1]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2 as cv

## 데이터셋 위치 지정 

In [3]:
DATA_FOLDER = '../datasets/deepfake-detection'
TRAIN_SAMPLE_FOLDER = 'train_sample_videos'

print(f"Train samples: {len(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))}")

Train samples: 401


## 데이터 분석하기

In [6]:
train_list = list(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))
ext_dict = []
for file in train_list:
    file_ext = file.split('.')[1]
    if (file_ext not in ext_dict):
        ext_dict.append(file_ext)
print(f"Extensions: {ext_dict}")

for file_ext in ext_dict:
    print(f"Files with extension `{file_ext}`: {len([file for file in train_list if  file.endswith(file_ext)])}")
    
    json_file = [file for file in train_list if  file.endswith('json')][0]
print(f"JSON file: {json_file}")

Extensions: ['mp4', 'json']
Files with extension `mp4`: 400
Files with extension `json`: 1
JSON file: metadata.json


영상들이 모여있는 mp4와 FAKE/REAL 라벨링 정보를 가지고 있는 json이 있는 것으로 보임.  그리고 그 json파일의 이름은 metadata.json

In [7]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, json_file))
    df = df.T
    return df

meta_train_df = get_meta_from_json(TRAIN_SAMPLE_FOLDER)
meta_train_df.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


metadata.json의 property 구성을 알 수 있다. 문제는 train데이터만 구성된 것을 볼 수 있다. 즉, train데이터를 나누어야 할 것 같다.

In [9]:
fake_train_sample_video = list(meta_train_df.loc[meta_train_df.label=='FAKE'].sample(3).index)
print(fake_train_sample_video)
real_train_sample_video = list(meta_train_df.loc[meta_train_df.label=='REAL'].sample(3).index)
print(real_train_sample_video)

['ahdbuwqxit.mp4', 'aipfdnwpoo.mp4', 'aknbdpmgua.mp4']
['aybgughjxh.mp4', 'dzyuwjkjui.mp4', 'bulkxhhknf.mp4']


REAL과 FAKE 영상은 이런식으로 뽑아오면 될 듯하다